In [1]:
import pickle, requests, folium, os, json

In [2]:
# pickle 파일 불러오기
with open('farm.p', 'rb') as f :
    loaded_dict = pickle.load(f)
print(loaded_dict)

{'꿈목장': '경기 김포시 통진읍 귀전리 940', '내화목장': '경남 양산시 원동면 화제리 826-1', '농도원목장': '경기도 용인시 처인구 원삼면 원양로377번길 1-34', '다래목장': '충북 청주시 청원구 내수읍 초정약수로 526-21', '대한목장': '전북 임실군 지사면 영천리 1295 ', '모산목장': '경기도 파주시 탄현면 검산로519번길 6-36', '미르목장': '전남 영광군 법성면 대덕리 57-27', '성동목장': '충남 부여군 임천면 두곡리 449', '신우목장': '울산광역시 울주군 두서면 아미산못길 34', '신하늘목장': '경기 화성 장덕동 273', '아침미소목장': '제주 제주시 월평동 157', '애심목장': '경기 연천 연천 와초리 74-4', '와우목장': '경기도 이천시 설성면 진상미로665번길 95-14', '요라파목장': '경남 사천시 곤양면 환덕리 70', '은아목장': '경기 여주시 가남읍 금당5길 139', '지헌목장': '제주 제주시 애월읍 어음 13길 16', '진주목장': '경기 화성시 비봉면 일묵동길 24', '청계목장': '경기 용인 처인구 백암면 박곡리 270-3', '청원목장': '충북 청주시 서원구 현도면 우록1길 91-3', '태신목장': '충남 예산군 고덕면 상몽리 442-1', '파도목장': '전라남도 무안군 현경면 해운로 185-10', '하늘목장': '강원도 평창군 대관령면 횡계리 산1-1', '해뜰목장': '경남 하동군 옥종면 양구리 산77'}


In [3]:
secret_file = os.path.join('../secrets.json')

with open(secret_file) as f:
    secrets = json.loads(f.read())

In [4]:
# API 요청
url = 'https://dapi.kakao.com/v2/local/search/address.json'
kakao_key = secrets['KAKAO_KEY']
headers = {'Authorization': 'KakaoAK ' + kakao_key}

map_center = [36.3547170088274, 127.765271596878]
my_map = folium.Map(location=map_center, zoom_start=7)

# 지도에 마커를 추가하는 함수
def add_marker(coord, name, num) :
    text = f'<h4><a href="detail/{num}" style="text-decoration: none; color: #333;">{name}</a></h4>'
    popup = folium.Popup(text, min_width=50, max_width=200)
    folium.Marker(location=coord, popup=popup).add_to(my_map)
    
# 각 주소를 순회하며 지도에 마커를 추가
result = {}
i = 1

for name, address in loaded_dict.items() :
    params = {'query': address}
    response = requests.get(url, headers=headers, params=params)
    
    if response.status_code == 200 :
        data = response.json()
        coordinates = data['documents'][0]['address']
        coord = [float(coordinates['y']), float(coordinates['x'])]
        add_marker(coord, name, i)
        i += 1

        result[name] = [address, coord]
    else :
        break

In [5]:
my_map

In [33]:
# 지도 시각화 저장
my_map.save("map.html")

In [10]:
with open('new_farm.p', 'wb') as f:
    pickle.dump(result, f)